![](./img/chinahadoop.png)

## 头条官方分享
**原文地址：[36氪首发 | 今日头条推荐算法原理全文详解](https://36kr.com/p/5114077)**

本次分享将主要介绍今日头条推荐系统概览以及内容分析、用户标签、评估分析，内容安全等原理。

如今，算法分发已经逐步成为信息平台、搜索引擎、浏览器、社交软件等几乎所有软件的标配，但同时也开始面临各种不同的质疑、挑战与误解。

2018年1月，今日头条资深算法架构师曹欢欢博士，首次公开今日头条的算法原理，以期推动整个行业问诊算法、建言算法。通过让算法透明，来消除各界对算法的误解。

据悉，今日头条的信息推荐算法自2012年9月第一版开发运行至今，已经经过四次大调整和修改。目前服务全球亿万用户。



**以下为曹欢欢关于《今日头条算法原理》的分享内容（已授权）：** 

![36氪首发 | 今日头条推荐算法原理全文详解](https://pic.36krcnd.com/avatar/201801/15154249/256gyo77nhy20jig.jpeg!1200)

本次分享将主要介绍今日头条推荐系统概览以及内容分析、用户标签、评估分析，内容安全等原理。

![36氪首发 | 今日头条推荐算法原理全文详解](https://pic.36krcnd.com/avatar/201801/15154249/egdblny7enpnm9vt.jpeg!1200)

### 一、系统概览 

推荐系统，如果用形式化的方式去描述实际上是拟合一个用户对内容满意度的函数，这个函数需要输入三个维度的变量。

![36氪首发 | 今日头条推荐算法原理全文详解](https://pic.36krcnd.com/avatar/201801/15154252/b8tvjj3zffaybojh.jpeg!1200)

**第一个维度是内容。**头条现在已经是一个综合内容平台，图文、视频、UGC小视频、问答、微头条，每种内容有很多自己的特征，需要考虑怎样提取不同内容类型的特征做好推荐。

**第二个维度是用户特征。**包括各种兴趣标签，职业、年龄、性别等，还有很多模型刻划出的隐式用户兴趣等。

**第三个维度是环境特征。**这是移动互联网时代推荐的特点，用户随时随地移动，在工作场合、通勤、旅游等不同的场景，信息偏好有所偏移。

结合三方面的维度，模型会给出一个预估，即推测推荐内容在这一场景下对这一用户是否合适。

这里还有一个问题，如何引入无法直接衡量的目标？

![36氪首发 | 今日头条推荐算法原理全文详解](https://pic.36krcnd.com/avatar/201801/15154253/1tru0qaqj0rayuk5.jpeg!1200)

推荐模型中，点击率、阅读时间、点赞、评论、转发包括点赞都是可以量化的目标，能够用模型直接拟合做预估，看线上提升情况可以知道做的好不好。但一个大体量的推荐系统，服务用户众多，不能完全由指标评估，引入数据指标以外的要素也很重要。

比如广告和特型内容频控。像问答卡片就是比较特殊的内容形式，其推荐的目标不完全是让用户浏览，还要考虑吸引用户回答为社区贡献内容。这些内容和普通内容如何混排，怎样控制频控都需要考虑。

此外，**平台出于内容生态和社会责任的考量，像低俗内容的打压，标题党、低质内容的打压，重要新闻的置顶、加权、强插，低级别账号内容降权都是算法本身无法完成，需要进一步对内容进行干预。** 

下面我将简单介绍在上述算法目标的基础上如何对其实现。

![36氪首发 | 今日头条推荐算法原理全文详解](https://pic.36krcnd.com/avatar/201801/15154253/akfjjicju89i66ar.jpeg!1200)前面提到的公式y = F(Xi ,Xu ,Xc)，是一个很经典的监督学习问题。可实现的方法有很多，比如传统的协同过滤模型，监督学习算法Logistic Regression模型，基于深度学习的模型，Factorization Machine和GBDT等。

一个优秀的工业级推荐系统需要非常灵活的算法实验平台，可以支持多种算法组合，包括模型结构调整。因为很难有一套通用的模型架构适用于所有的推荐场景。现在很流行将LR和DNN结合，前几年Facebook也将LR和GBDT算法做结合。今日头条旗下几款产品都在沿用同一套强大的算法推荐系统，但根据业务场景不同，模型架构会有所调整。 

![36氪首发 | 今日头条推荐算法原理全文详解](https://pic.36krcnd.com/avatar/201801/15154252/2ufgveuh9kalwd42.jpeg!1200)

模型之后再看一下典型的推荐特征，主要有四类特征会对推荐起到比较重要的作用。

**第一类是相关性特征，就是评估内容的属性和与用户是否匹配。**显性的匹配包括关键词匹配、分类匹配、来源匹配、主题匹配等。像FM模型中也有一些隐性匹配，从用户向量与内容向量的距离可以得出。

**第二类是环境特征，包括地理位置、时间。这些既是bias特征，也能以此构建一些匹配特征。**

**第三类是热度特征。**包括全局热度、分类热度，主题热度，以及关键词热度等。内容热度信息在大的推荐系统特别在用户冷启动的时候非常有效。

**第四类是协同特征，它可以在部分程度上帮助解决所谓算法越推越窄的问题。**协同特征并非考虑用户已有历史。而是通过用户行为分析不同用户间相似性，比如点击相似、兴趣分类相似、主题相似、兴趣词相似，甚至向量相似，从而扩展模型的探索能力。 

![36氪首发 | 今日头条推荐算法原理全文详解](https://pic.36krcnd.com/avatar/201801/15154253/ctle81m5pnghemqo.jpeg!1200)

模型的训练上，头条系大部分推荐产品采用实时训练。实时训练省资源并且反馈快，这对信息流产品非常重要。用户需要行为信息可以被模型快速捕捉并反馈至下一刷的推荐效果。我们线上目前基于storm集群实时处理样本数据，包括点击、展现、收藏、分享等动作类型。模型参数服务器是内部开发的一套高性能的系统，因为头条数据规模增长太快，类似的开源系统稳定性和性能无法满足，而我们自研的系统底层做了很多针对性的优化，提供了完善运维工具，更适配现有的业务场景。

目前，头条的推荐算法模型在世界范围内也是比较大的，包含几百亿原始特征和数十亿向量特征。整体的训练过程是线上服务器记录实时特征，导入到Kafka文件队列中，然后进一步导入Storm集群消费Kafka数据，客户端回传推荐的label构造训练样本，随后根据最新样本进行在线训练更新模型参数，最终线上模型得到更新。这个过程中主要的延迟在用户的动作反馈延时，因为文章推荐后用户不一定马上看，不考虑这部分时间，整个系统是几乎实时的。

![36氪首发 | 今日头条推荐算法原理全文详解](https://pic.36krcnd.com/avatar/201801/15154253/8yitwoytgmtnd168.jpeg!1200)

但因为头条目前的内容量非常大，加上小视频内容有千万级别，推荐系统不可能所有内容全部由模型预估。所以需要设计一些召回策略，每次推荐时从海量内容中筛选出千级别的内容库。召回策略最重要的要求是性能要极致，一般超时不能超过50毫秒。 

![36氪首发 | 今日头条推荐算法原理全文详解](https://pic.36krcnd.com/avatar/201801/15154253/ar1sm8q55u919pc3.jpeg!1200)

召回策略种类有很多，我们主要用的是倒排的思路。离线维护一个倒排，这个倒排的key可以是分类，topic，实体，来源等，排序考虑热度、新鲜度、动作等。线上召回可以迅速从倒排中根据用户兴趣标签对内容做截断，高效的从很大的内容库中筛选比较靠谱的一小部分内容。 

![36氪首发 | 今日头条推荐算法原理全文详解](https://pic.36krcnd.com/avatar/201801/15154253/url1prwqngvrjx0f.jpeg!1200)

### 二、内容分析

内容分析包括文本分析，图片分析和视频分析。头条一开始主要做资讯，今天我们主要讲一下文本分析。文本分析在推荐系统中一个很重要的作用是用户兴趣建模。没有内容及文本标签，无法得到用户兴趣标签。举个例子，只有知道文章标签是互联网，用户看了互联网标签的文章，才能知道用户有互联网标签，其他关键词也一样。

![36氪首发 | 今日头条推荐算法原理全文详解](https://pic.36krcnd.com/avatar/201801/15160517/rqnahcdwtmv1xzez.jpeg!1200)另一方面，文本内容的标签可以直接帮助推荐特征，比如魅族的内容可以推荐给关注魅族的用户，这是用户标签的匹配。如果某段时间推荐主频道效果不理想，出现推荐窄化，用户会发现到具体的频道推荐（如科技、体育、娱乐、军事等）中阅读后，再回主feed,推荐效果会更好。因为整个模型是打通的，子频道探索空间较小，更容易满足用户需求。只通过单一信道反馈提高推荐准确率难度会比较大，子频道做的好很重要。而这也需要好的内容分析。

![36氪首发 | 今日头条推荐算法原理全文详解](https://pic.36krcnd.com/avatar/201801/15160538/rnky59il2od10fn9.jpeg!1200)

上图是今日头条的一个实际文本case。可以看到，这篇文章有分类、关键词、topic、实体词等文本特征。当然不是没有文本特征，推荐系统就不能工作，推荐系统最早期应用在Amazon,甚至沃尔玛时代就有，包括Netfilx做视频推荐也没有文本特征直接协同过滤推荐。**但对资讯类产品而言，大部分是消费当天内容，没有文本特征新内容冷启动非常困难，协同类特征无法解决文章冷启动问题。**

![36氪首发 | 今日头条推荐算法原理全文详解](https://pic.36krcnd.com/avatar/201801/15160549/1o62umwiaef0gado.jpeg!1200)今日头条推荐系统主要抽取的文本特征包括以下几类。首先是语义标签类特征，显式为文章打上语义标签。这部分标签是由人定义的特征，每个标签有明确的意义，标签体系是预定义的。此外还有隐式语义特征，主要是topic特征和关键词特征，其中topic特征是对于词概率分布的描述，无明确意义；而关键词特征会基于一些统一特征描述，无明确集合。

![36氪首发 | 今日头条推荐算法原理全文详解](https://pic.36krcnd.com/avatar/201801/15160643/qr5rt30sjdud81eq.jpeg!1200)另外文本相似度特征也非常重要。在头条，曾经用户反馈最大的问题之一就是为什么总推荐重复的内容。这个问题的难点在于，每个人对重复的定义不一样。举个例子，有人觉得这篇讲皇马和巴萨的文章，昨天已经看过类似内容，今天还说这两个队那就是重复。但对于一个重度球迷而言，尤其是巴萨的球迷，恨不得所有报道都看一遍。解决这一问题需要根据判断相似文章的主题、行文、主体等内容，根据这些特征做线上策略。

同样，还有时空特征，分析内容的发生地点以及时效性。比如武汉限行的事情推给北京用户可能就没有意义。最后还要考虑质量相关特征，判断内容是否低俗，色情，是否是软文，鸡汤？ 

![36氪首发 | 今日头条推荐算法原理全文详解](https://pic.36krcnd.com/avatar/201801/15160711/d0ufcrvlkdf6a0tx.jpeg!1200)

上图是头条语义标签的特征和使用场景。他们之间层级不同，要求不同。

![36氪首发 | 今日头条推荐算法原理全文详解](https://pic.36krcnd.com/avatar/201801/15160734/cazc3cklyyd66eic.jpeg!1200)分类的目标是覆盖全面，希望每篇内容每段视频都有分类；而实体体系要求精准，相同名字或内容要能明确区分究竟指代哪一个人或物，但不用覆盖很全。概念体系则负责解决比较精确又属于抽象概念的语义。这是我们最初的分类，实践中发现分类和概念在技术上能互用，后来统一用了一套技术架构。

![36氪首发 | 今日头条推荐算法原理全文详解](https://pic.36krcnd.com/avatar/201801/15160757/glup00orydf53l3j.png!1200)目前，隐式语义特征已经可以很好的帮助推荐，而语义标签需要持续标注，新名词新概念不断出现，标注也要不断迭代。其做好的难度和资源投入要远大于隐式语义特征，那为什么还需要语义标签？有一些产品上的需要，比如频道需要有明确定义的分类内容和容易理解的文本标签体系。语义标签的效果是检查一个公司NLP技术水平的试金石。

![36氪首发 | 今日头条推荐算法原理全文详解](https://pic.36krcnd.com/avatar/201801/15160820/93sw0u30n1l16onv.jpeg!1200)

今日头条推荐系统的线上分类采用典型的层次化文本分类算法。最上面Root，下面第一层的分类是像科技、体育、财经、娱乐，体育这样的大类，再下面细分足球、篮球、乒乓球、网球、田径、游泳...，足球再细分国际足球、中国足球，中国足球又细分中甲、中超、国家队...，相比单独的分类器，利用层次化文本分类算法能更好地解决数据倾斜的问题。有一些例外是，如果要提高召回，可以看到我们连接了一些飞线。这套架构通用，但根据不同的问题难度，每个元分类器可以异构，像有些分类SVM效果很好，有些要结合CNN，有些要结合RNN再处理一下。

![36氪首发 | 今日头条推荐算法原理全文详解](https://pic.36krcnd.com/avatar/201801/15160833/mecnwdsmk1puqtdd.jpeg!1200)

上图是一个实体词识别算法的case。基于分词结果和词性标注选取候选，期间可能需要根据知识库做一些拼接，有些实体是几个词的组合，要确定哪几个词结合在一起能映射实体的描述。如果结果映射多个实体还要通过词向量、topic分布甚至词频本身等去歧，最后计算一个相关性模型。

### 三、用户标签

内容分析和用户标签是推荐系统的两大基石。内容分析涉及到机器学习的内容多一些，相比而言，用户标签工程挑战更大。

![36氪首发 | 今日头条推荐算法原理全文详解](https://pic.36krcnd.com/avatar/201801/15160845/wjh4czz54udvcj5y.jpeg!1200)

今日头条常用的用户标签包括用户感兴趣的类别和主题、关键词、来源、基于兴趣的用户聚类以及各种垂直兴趣特征（车型，体育球队，股票等）。还有性别、年龄、地点等信息。性别信息通过用户第三方社交账号登录得到。年龄信息通常由模型预测，通过机型、阅读时间分布等预估。常驻地点来自用户授权访问位置信息，在位置信息的基础上通过传统聚类的方法拿到常驻点。常驻点结合其他信息，可以推测用户的工作地点、出差地点、旅游地点。这些用户标签非常有助于推荐。

![36氪首发 | 今日头条推荐算法原理全文详解](https://pic.36krcnd.com/avatar/201801/15160932/2gpefax7ni2jbfd0.jpeg!1200)当然最简单的用户标签是浏览过的内容标签。但这里涉及到一些数据处理策略。主要包括：一、过滤噪声。通过停留时间短的点击，过滤标题党。二、热点惩罚。对用户在一些热门文章（如前段时间PG One的新闻）上的动作做降权处理。理论上，传播范围较大的内容，置信度会下降。三、时间衰减。用户兴趣会发生偏移，因此策略更偏向新的用户行为。因此，随着用户动作的增加，老的特征权重会随时间衰减，新动作贡献的特征权重会更大。四、惩罚展现。如果一篇推荐给用户的文章没有被点击，相关特征（类别，关键词，来源）权重会被惩罚。当然同时，也要考虑全局背景，是不是相关内容推送比较多，以及相关的关闭和dislike信号等。

![36氪首发 | 今日头条推荐算法原理全文详解](https://pic.36krcnd.com/avatar/201801/15161003/6mkmx5tticrdpqnx.jpeg!1200)

用户标签挖掘总体比较简单，主要还是刚刚提到的工程挑战。头条用户标签第一版是批量计算框架，流程比较简单，每天抽取昨天的日活用户过去两个月的动作数据，在Hadoop集群上批量计算结果。

![36氪首发 | 今日头条推荐算法原理全文详解](https://pic.36krcnd.com/avatar/201801/15161017/8jtrq29m33v0lh7a.jpeg!1200)但问题在于，随着用户高速增长，兴趣模型种类和其他批量处理任务都在增加，涉及到的计算量太大。2014年，批量处理任务几百万用户标签更新的Hadoop任务，当天完成已经开始勉强。集群计算资源紧张很容易影响其它工作，集中写入分布式存储系统的压力也开始增大，并且用户兴趣标签更新延迟越来越高。 

![36氪首发 | 今日头条推荐算法原理全文详解](https://pic.36krcnd.com/avatar/201801/15161044/z09fwoiebsm5b2rz.jpeg!1200)面对这些挑战。2014年底今日头条上线了用户标签Storm集群流式计算系统。改成流式之后，只要有用户动作更新就更新标签，CPU代价比较小，可以节省80%的CPU时间，大大降低了计算资源开销。同时，只需几十台机器就可以支撑每天数千万用户的兴趣模型更新，并且特征更新速度非常快，基本可以做到准实时。这套系统从上线一直使用至今。

![36氪首发 | 今日头条推荐算法原理全文详解](https://pic.36krcnd.com/avatar/201801/15161144/sn7dut238ovnse25.jpeg!1200)当然，我们也发现并非所有用户标签都需要流式系统。像用户的性别、年龄、常驻地点这些信息，不需要实时重复计算，就仍然保留daily更新。

### 四、评估分析

上面介绍了推荐系统的整体架构，那么如何评估推荐效果好不好？

有一句我认为非常有智慧的话，“一个事情没法评估就没法优化”。对推荐系统也是一样。 

![36氪首发 | 今日头条推荐算法原理全文详解](https://pic.36krcnd.com/avatar/201801/15161210/01wbic94xodxlegl.jpeg!1200)

事实上，很多因素都会影响推荐效果。比如侯选集合变化，召回模块的改进或增加，推荐特征的增加，模型架构的改进在，算法参数的优化等等，不一一举例。评估的意义就在于，很多优化最终可能是负向效果，并不是优化上线后效果就会改进。

![36氪首发 | 今日头条推荐算法原理全文详解](https://pic.36krcnd.com/avatar/201801/15161222/31su8zjhcessopzf.jpeg!1200)

全面的评估推荐系统，需要完备的评估体系、强大的实验平台以及易用的经验分析工具。所谓完备的体系就是并非单一指标衡量，不能只看点击率或者停留时长等，需要综合评估。过去几年我们一直在尝试，能不能综合尽可能多的指标合成唯一的评估指标，但仍在探索中。目前，我们上线还是要由各业务比较资深的同学组成评审委员会深入讨论后决定。

很多公司算法做的不好，并非是工程师能力不够，而是需要一个强大的实验平台，还有便捷的实验分析工具，可以智能分析数据指标的置信度。

![36氪首发 | 今日头条推荐算法原理全文详解](https://pic.36krcnd.com/avatar/201801/15161239/265ji8lu87ryswq0.jpeg!1200)

一个良好的评估体系建立需要遵循几个原则，首先是兼顾短期指标与长期指标。我在之前公司负责电商方向的时候观察到，很多策略调整短期内用户觉得新鲜，但是长期看其实没有任何助益。

其次，要兼顾用户指标和生态指标。今日头条作为内容分创作平台，既要为内容创作者提供价值，让他更有尊严的创作，也有义务满足用户，这两者要平衡。还有广告主利益也要考虑，这是多方博弈和平衡的过程。 

另外，要注意协同效应的影响。实验中严格的流量隔离很难做到，要注意外部效应。

![36氪首发 | 今日头条推荐算法原理全文详解](https://pic.36krcnd.com/avatar/201801/15161259/ud7dpj86brl4jv0z.jpeg!1200)

强大的实验平台非常直接的优点是，当同时在线的实验比较多时，可以由平台自动分配流量，无需人工沟通，并且实验结束流量立即回收，提高管理效率。这能帮助公司降低分析成本，加快算法迭代效应，使整个系统的算法优化工作能够快速往前推进。

![36氪首发 | 今日头条推荐算法原理全文详解](https://pic.36krcnd.com/avatar/201801/15161312/rhj46pz2fzahbd5s.jpeg!1200)

这是头条A/B Test实验系统的基本原理。首先我们会做在离线状态下做好用户分桶，然后线上分配实验流量，将桶里用户打上标签，分给实验组。举个例子，开一个10%流量的实验，两个实验组各5%，一个5%是基线，策略和线上大盘一样，另外一个是新的策略。

![36氪首发 | 今日头条推荐算法原理全文详解](https://pic.36krcnd.com/avatar/201801/15161324/qhjs16a7aiz7d751.jpeg!1200)

实验过程中用户动作会被搜集，基本上是准实时，每小时都可以看到。但因为小时数据有波动，通常是以天为时间节点来看。动作搜集后会有日志处理、分布式统计、写入数据库，非常便捷。

![36氪首发 | 今日头条推荐算法原理全文详解](https://pic.36krcnd.com/avatar/201801/15161335/dwvzv5xx8znumu0v.jpeg!1200)

在这个系统下工程师只需要设置流量需求、实验时间、定义特殊过滤条件，自定义实验组ID。系统可以自动生成：实验数据对比、实验数据置信度、实验结论总结以及实验优化建议。

![36氪首发 | 今日头条推荐算法原理全文详解](https://pic.36krcnd.com/avatar/201801/15161347/fxsf60willba3mjo.jpeg!1200)

当然，只有实验平台是远远不够的。线上实验平台只能通过数据指标变化推测用户体验的变化，但数据指标和用户体验存在差异，很多指标不能完全量化。很多改进仍然要通过人工分析，重大改进需要人工评估二次确认。

### 五、内容安全

![36氪首发 | 今日头条推荐算法原理全文详解](https://pic.36krcnd.com/avatar/201801/15161400/rbxltgcd7fotaj6h.jpeg!1200)最后要介绍今日头条在内容安全上的一些举措。头条现在已经是国内最大的内容创作与分发凭条，必须越来越重视社会责任和行业领导者的责任。如果1%的推荐内容出现问题，就会产生较大的影响。 

因此头条从创立伊始就把内容安全放在公司最高优先级队列。成立之初，已经专门设有审核团队负责内容安全。当时研发所有客户端、后端、算法的同学一共才不到40人，头条非常重视内容审核。

![36氪首发 | 今日头条推荐算法原理全文详解](https://pic.36krcnd.com/avatar/201801/15161412/27kngjfye7x15zmb.jpeg!1200)

现在，今日头条的内容主要来源于两部分，一是具有成熟内容生产能力的PGC平台

一是UGC用户内容，如问答、用户评论、微头条。这两部分内容需要通过统一的审核机制。如果是数量相对少的PGC内容，会直接进行风险审核，没有问题会大范围推荐。UGC内容需要经过一个风险模型的过滤，有问题的会进入二次风险审核。审核通过后，内容会被真正进行推荐。这时如果收到一定量以上的评论或者举报负向反馈，还会再回到复审环节，有问题直接下架。整个机制相对而言比较健全，作为行业领先者，在内容安全上，今日头条一直用最高的标准要求自己。

![36氪首发 | 今日头条推荐算法原理全文详解](https://pic.36krcnd.com/avatar/201801/15161424/0m8scz8iaq4h1saf.jpeg!1200)

分享内容识别技术主要鉴黄模型，谩骂模型以及低俗模型。今日头条的低俗模型通过深度学习算法训练，样本库非常大，图片、文本同时分析。这部分模型更注重召回率，准确率甚至可以牺牲一些。谩骂模型的样本库同样超过百万，召回率高达95%+，准确率80%+。如果用户经常出言不讳或者不当的评论，我们有一些惩罚机制。

![36氪首发 | 今日头条推荐算法原理全文详解](https://pic.36krcnd.com/avatar/201801/15161439/twcp7w6i7z6ym1dk.jpeg!1200)

泛低质识别涉及的情况非常多，像假新闻、黑稿、题文不符、标题党、内容质量低等等，这部分内容由机器理解是非常难的，需要大量反馈信息，包括其他样本信息比对。目前低质模型的准确率和召回率都不是特别高，还需要结合人工复审，将阈值提高。目前最终的召回已达到95%，这部分其实还有非常多的工作可以做。头条人工智能实验室李航老师目前也在和密歇根大学共建科研项目，设立谣言识别平台。

以上是头条推荐系统的原理分享，希望未来得到更多的建议，帮助我们更好改进工作。

## 部分解读

### 第1点

【原文】今日头条算法推荐系统，主要输入三个维度的变量。一是内容特征，图文、视频、UGC小视频、问答、微头条等，每种内容有很多自己的特征，需要分别提取；二是用户特征，包括兴趣标签、职业、年龄、性别、机型等，以及很多模型刻画出的用户隐藏兴趣。三是环境特征，不同的时间不同的地点不同的场景（工作/通勤/旅游等），用户对信息的偏好有所不同。结合这三方面纬度，今日头条的推荐模型做预估，这个内容在这个场景下对这个用户是否合适。

【解析】：

- （1）内容特征，好理解，既是对各类文章、视频提取关键要素，比如可以通过对文本进行语义识别，识别出文章的主要关键字，视频标题关键字如何，这个工作主要目的是对文章进行画像，方便以后对客户推荐。至于能否做到视频内容的关键字提取，比如虽然标题是周杰伦相关视频，但视频内容全部是方文山的帅气身影，能否将视频中方文山这个关键字提取出来，则暂不可知，这个工作需要耗费巨大服务器资源，估计头条暂时还没有做吧。
- （2）用户特征，同上面内容特征一样，提取你的有效数据，比如经常浏览哪种类型文字、你经常搜索的关键字、你注册时登记信息的内容、还有原文提到的用什么手机之类，你是iphone我便显示这个游戏只能iphone玩，你是安卓我便显示这个游戏只能安卓玩、甚至精细的话可以将每个人的留言呀、评论呀等进行关键字提取，比如你经常评论历史内容文章，即使你浏览此类文章比较少，但是也可以作为一个关键指标进行使用，通过以上各类方式最终形成用户画像，方便后面对你进行文章视频推送。举个例子，比如在通信行业，我们经常会受到短信，推荐什么什么套餐、购买流量包之类，这就是通信公司通过客户分群、客户画像来实现的定向推送，以保证推送效果、节省推送成本，通信行业对一个人的画像会多达上千个关键指标，用户特征的提取对推荐的准确度、有效度会起到相当大的作用；
    - 注：画像是指对一篇文章或一个人的关键点分析，通过这些关键点则可大体了解一篇文章或者一个人的具体形象。
- （3）环境特征，比如获取你当前所在位置是否在旅游区呀，这个可以通过获取你实时位置来实现，然后与你之前经常出现的所在地进行对比等方式确认当前状态，是在家还是在旅游。如果系统检测到你在泰山游玩，则可能会相应推送泰山的类似文章资料等等，让你感觉：哇，头条好智能！

通过上面三个方面可以作为数据基础，分析当前你处于什么环境下，结合你的用户画像以及文章的画像来推荐，尽量做到推送给你的内容都是你感兴趣的。

### 第2点

【原文】点击率、阅读时间、点赞、评论、转发，这些都是可以量化的。但一个大体量的推荐系统，服务用户众多，不能完全由指标评估，引入数据以外的要素，也很重要。有些算法可以完成，有些算法还做不到、做的不好，这就需要内容干预。

【解析】：可以量化的指标可以体现一些具体信息，但是完全靠这个并不全面，比如刷评论、转发等情况，可能就会迷惑后台计算机的分析，纯粹靠这些数据进行推荐是有问题，里面提到的【数据以外的要素】，比如当前社会热点，虽然你的文章很热，但是热点已过，也不会因为文章的点击率大而继续给你推送；还有国家政策形势呀，国家政策如有调整，你的文章内容方向已不符合则也不会再次推荐，例如房产方面文章等；上述这些数据并不存在于推荐算法数据里，所以算法无法自动完成，所以需要人工对这些数据内容进行维护设置，以完善推荐机制。

### 第3点

【原文】没有一套通用的模型架构，适用所有的推荐场景。我们需要一个非常灵活的算法实验平台，这个算法不行，马上试另一个算法，实际上是各种算法的一个复杂组合。西瓜视频、火山小视频、抖音短视频、悟空问答，都在用头条这一套推荐系统，但具体到每套系统，架构都不一样，需要不断去试。

【解析】不同推荐场景需要不同的模型架构，这个是比较容易理解的，比如小视频，是短时投入，客户会很少计较时间成本，所以关键的推荐机制可能并不是你的用户画像特征，而是根据视频的点击率进行推荐，点击率越大的视频肯定越容易吸引用户的兴趣；而文章需要投入时间成本较高，精准投送要求会比较高，所以各类内容推荐算法会根据不同的特点进行不一样的推荐机制，需要不断尝试优化。

算法实验平台，好理解，比如第一种算法投入测试，推送10000条信息，实际点击有效率是50%，相当于一条测试数据；后面再对其他多种算法测试，最终选取点击率最高的算法进行应用。

### 第4点

【原文】算法推荐要达到不错的效果，需要解决好这四类特征：相关性特征、环境特征、热度特征和协同特征。相关性特征，解决内容和用户的匹配。环境特征，解决基础特征和匹配。热度特征，在冷启动上很有效。协同特征，考虑相似用户的兴趣，在一定程度上解决所谓算法越推越窄的问题。

【解析】相关性特征，即是文章画像与你的用户画像一致性比较高，则进行相应推荐。环境特征，刚才也已经说明。热度特征，在冷启动很有效，冷启动的概念是说系统里面并没有你的相关画像数据，比如游客登录了头条，会怎么推荐呢，可以通过热度特征，即根据当前社会热点呀、文章阅读量高低排名呀等一些能代表普遍群体感受的指标进行推送选择。协同特征，后面解释了，为了解决算法越推越窄的问题，比如通过你的行为、画像、环境等各类指标进行模型分析后，适合推荐给你的文章太少，应该怎么办呢，这时候就可以寻找跟你特征相同的用户，比如跟你同一个年龄段、同一个性别、同一个地区等相似用户的兴趣给你进行推荐，可以作为一个推荐的补充，以实现比较好的推荐效果。

### 第5点

【原文】今日头条有一个世界范围内比较大的在线训练推荐模型，包括几百亿特征和几十亿的向量特征。完全依赖模型推荐成本过高，因此有了简化策略的召回模型。基于召回策略，把一个海量、无法把握的内容库，变成一个相对小、可以把握的内容库，再进入推荐模型。这样有效平衡了计算成本和效果。

【解析】：这几百亿特征，大家可能很难想象，主要就是包括上篇所说的文章画像、视频画像、用户画像，所有文章包括普通文章、专业文章，所有关键词集中起来数量是相当可观的，这每一个关键字，在拟合算法中就是一个向量。所以如果每一次的推荐都依赖这个最大的模型，成本确实太高，所以需要简化。

说明后面内容之前，需要介绍一个关键词的含义：召回，说直接点，就是对符合条件的结果进行排序，以决定给你推荐文章的顺序。在业界对推荐算法是否有效进行评价的时候，需要用到召回率这个指标，举例：所有可供推荐的文章库里共有40篇文章与你的用户画像相关，实际通过推荐算法推荐给你了20篇，但实际上有10篇是你真正喜欢的文章，则召回率就是10/40=0.25，精度是10/20=0.5。整句话意思就是根据推荐算法尽量缩小推荐内容库，减少计算推荐成本。

### 第6点

【原文】在今日头条工作前三年，我收到用户反馈最大的一个问题，就是，“怎么老给我推重复的？” 其实，每个人对重复的定义不一样。有人昨天看到一篇讲巴萨的文章，今天又看到两篇，可能就觉得烦了。但对于一个重度球迷来讲，比如巴萨的球迷，可能恨不得所有的报道都看一遍。解决这个，实际上需要精确抽取文本特征，比如哪些文章说的是一个事儿，哪些文章基本一样等等。文本特征对于推荐的独特价值在于，没有文本特征，推荐引擎无法工作，同时，文本特征颗粒度越细，冷启动能力越强。

【解析】：文本特征提取本身也是一个比较复杂的问题，举个例子，要对”人民生活水平”这句话进行语义解析提取特征，可能会提取出如下一些关键字：人民-民生-生活-活水-水平，至于怎么提取才贴合本篇文章要表达的意思，则需要很复杂的后台逻辑了，比如可以随便摘取人民日报1000篇文章作为语料库，分析各个词语出现的频率，把这个词语频率高低排序作为算法的基础数据来进行本篇文章文本特征的提取，这是一种比较简单的方式。具体的要跟本篇文章如何结合，保证提取的特征是准确的则需要更复杂的算法。

冷启动的意思上篇大体说过，就是在没有用户信息的时候如何进行内容推荐，比如新注册的用户或者游客，比如腾讯公司统一用QQ号作为一个用户主索引，你玩过什么游戏，听过什么歌曲，都可以作为用户画像特征应用到腾讯新闻的对你推荐文章的算法里，头条当然也可以用sina微博账号、微信账号登录时的授权，分析你曾经关注的微博账号、微信公账号信息等，作为冷启动时的用户画像数据。

### 第7点

【原文】语义标签的效果，是检查一个公司NLP（自然语言处理）的试金石。频道、兴趣表达等重要产品功能，需要一个有明确定义、容易理解的文本标签体系。所以，在隐式语义特征已经可以很好地帮助推荐，且做好语义标签需要投入远大于隐式语义特征的情况下，我们仍然需要做好语义标签。

【解析】：何为隐式语义特征呢？简单说明一下，当两个词或一组词出现在同一个文档中时，这些词之间可以被认为是语义相关的。机器并不知道某个词究竟代表什么，不知道某个词是什么意思，但是通过大量文章的学习，机器就能学习出这些词语是语义相关，比如apple跟苹果、电脑和计算机，通过这些相关性就可以对你搜索的内容、或者对你的用户画像进行匹配来推荐文章，你搜电脑，则文章特征包含计算机的文章也会推荐给你。语义标签则是明确指定词语的语义、相关性作为推荐算法的基础数据来使用，也就是让机器明确明白词语的相关性，当然后面这种算法需要耗费相当大的人力物力，但是这个工作做好之后，就保证了数据的准确性，减少了依赖于机器学习的不可控制性，当然这个功能也是一个公司自然语言处理最能展示实力的一环。

### 第8点

【原文】除了用户的自然标签，推荐还需要考虑很多复杂的情况： 1）过滤噪声：过滤停留时间短的点击，打击标题党；2）惩罚热点：用户在热门文章上的动作做降权处理；3）时间衰减：随着用户动作的增加，老的特征权重会随时间衰减，新动作贡献的特征权重会更大；4）惩罚展现：如果一篇推荐给用户的文章没有被点击，相关特征（类别、关键词、来源）权重会被惩罚；5）考虑全局背景：考虑给定特征的人均点击比例。

【解析】：里面描述的内容主要是考虑用户的自然标签之外的一些因素，1）过滤噪声，一般在做信号处理的时候，需要对信号进行降噪处理，以平滑信号曲线方便进行下一步处理。此处的过滤停留时间短的点击，也是出于此考虑，因为标题党可能会引入比较大的点击率，但是也会有非常大的概率是进入后直接退出，则这些点击率并不能代表你的文章比较好，所以这部分点击就要过滤掉，所以这个功能如果真正在推荐算法中应用的话，标题党后面将不会再是提高收益的一种有效方式。
- 惩罚热点，每个人在热点上的表现可能仅仅是因为随大流，并不能代表你真正的用户画像，所以这部分要降低操作的权重，很好理解。
- 时间衰减，也好理解，比如你一年前喜欢看的文章特征，相对于你前几天喜欢的文章特征，当然是以前几天的特征为准作为推送依据啦。
- 惩罚展现，这就是为什么文章说不行一直不行，说行就猛地飙升上去的原因了。如果你写的一篇文章推荐给100个人，没有一个人点击，则你的文章特征权重就会降低，以后的推荐数量就会降低了，假设头条仁义的话，可以给你重新进行文章画像，再次进入推荐序列，如果不仁义则可能就直接打入冷宫，永不得翻身了。
- 考虑全局背景，相当于上面所说的降噪处理，比如你这篇文章一个人又评价又赞又转发，但是其他100个人冷眼旁观，根本不打开看，则人均点击比例也是非常低的，所以考虑这个因素的话你的文章也是不会得到大批量推荐的。

### 第9点

【原文】比起批量计算用户标签，采用流式计算框架，可以大大节省计算机资源，可以准实时完成用户兴趣模型的更新。几十台机器就可以支撑每天数千万用户的兴趣模型更新，99%的用户可以实现发生动作后10分钟模型更新。

【解析】这个地方了解批量处理跟流式框架即可，一般大数据处理工作，比如这里面说的批量计算用户标签，会选择业务量小的时间点进行批量处理，比如在凌晨，这样可以减少数据处理给业务带来的压力，统一更新用户的标签，但这里存在的一个问题便是无法及时更新用户的标签。
 比如你昨天看了很多足球的相关文章，足球这个标签的权重会相当高，今天上午突发奇想对篮球感兴趣，但如果标签更新延迟时间很长的话，可能下午系统才会反应过来给你推荐篮球的内容，你一上午的时间推送的可能全部是昨天关注的足球的内容，这肯定不是用户希望的。
 所以头条采用流式框架后，可以基本做到实时数据处理，基本在用户查看文章动作十分钟后就能对你的标签进行更新，提高用户感受。

### 第10点

【原文】影响推荐效果的因素有很多，我们需要一个完备的评估体系，不能只看单一指标，点击率、留存、收入或是互动，我们需要看很多指标，做综合评估：兼顾短期指标和长期指标，兼顾用户指标和生态指标，注意协同效应的影响，有时候需要做彻底的统计隔离等。有人问，所有的这些指标，能合成唯一的一个公式吗？我们苦苦探索了几年，目前还没有做到。

【解析】这里跟第二条类似，单纯靠具体统计指标来进行推荐效果肯定是不好的，比如单纯看一篇文章的点击量大，就推送给所有用户肯定是不合适的，所以需要进行综合评估。

这个综合评估，就需要对一些因素的有效期限进行分析，比如点击率有效期是比较短的，一篇文章在发布第一天点击率高则系统会疯狂推送，但是第二天可能点击率仍然比较高，但是跟一些热点性的因素相比，或者跟其他人写的新文章这个新鲜因素相比，这个点击率有效性就比较差了，可能就不会再次大批量推送，所以说一个有效的兼容性广的评估是需要进行全面设计的，具体指标的权重应该怎么设置比较好则需要通过不同的实验来判断。

### 第11点

【原文】很多公司的算法做得不好，不是人的问题，是实验平台的问题。 如果A/B Test，每次数据都是错的，不是这儿错就是那儿错，总上不了线，这个事就废了。而一个强大的实验平台，可以实现每天数百个实验同时在线，高效管理和分配实验流量，降低实验分析成本，提高算法迭代效率。

【解析】算法是需要测试出来的，数学模型里有各种各样的算法，其实每一种算法无所谓好坏，只是说在哪种场景下这个算法表现是比较好那个算法表现一般而已。如果一种算法机制能在概要设计阶段就能完全确定这样就是最好的话，就不存在实验的说法了，正是因为有很多不确定性，一种算法的优劣也是不确定的，所以能够降低试验成本、提高算法更新频率就成了提取有效算法的最主要的工作内容。

### 第12点

【原文】头条现在拥有健全的内容安全机制。除了人工审核团队，我们还有技术识别。包括风险内容识别技术，构建千万张图片样本集的鉴黄模型，超过百万样本库的低俗模型和谩骂模型等，以及泛低质内容识别技术。

【解析】这个比较容易理解了，随着头条作者数量增加，纯粹靠人工审核，那人力成本就太大了，所以机器智能识别的重要性就凸显出来，随着服务器性能的增加，一些重复性的工作就可以通过机器来实现，比如里面说的鉴黄模型，就是通过图片的相似性判断来让机器自动识别，也就是通过机器进行图像处理；

低俗模型和谩骂模型，主要是需要先建立一些文字模板公式，比如一句话里面含有哪些字符，或者哪些字符组合就定义为低俗模型，然后机器自动对文本进行匹配识别，当匹配上之后就监测出来进行过滤，这里面用到的主要是文本解析技术和语义识别技术，现在业界也都已经基本成熟，只是根据需要的不同进行不同的调整即可。